In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sagemaker.pytorch import PyTorch
from sagemaker.tensorflow import TensorFlow
import os

boto_session = boto3.Session(region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

role = "arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928"
role_name = role.split('/')[-1]  # Extract just the role name from the ARN

# Attach AdministratorAccess policy to your existing role
iam_client = boto3.client('iam')
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AdministratorAccess"
)
print(f"Attached AdministratorAccess policy to role: {role}")

# Assume blood.csv is in S3 already - if not, upload it first
input_data_s3_uri = "s3://blue-blood-data/final_df.csv"

bucket_name = "blue-blood-data"
region = 'us-east-1'
file_key = 'final_df.csv'




/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/26/25 16:14:30] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=280375;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=822621;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ubaid/Library/Application Support/sagemaker/config.yaml


[03/26/25 16:14:31] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=199313;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=617893;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=797097;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=890780;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

Attached AdministratorAccess policy to role: arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928


In [2]:
# First, make sure train.py exists in the notebook directory
if not os.path.exists("train.py"):
    print("Error: train.py not found in the current directory")
else:
    print("Found train.py in the current directory")
    
    # Upload train.py to S3 to ensure latest version is used
    code_prefix = "code"
    s3_code_path = sagemaker_session.upload_data("train.py", bucket=bucket_name, key_prefix=code_prefix)
    print(f"Uploaded train.py to {s3_code_path}")

# Verify that the S3 data file exists
try:
    s3_client = boto3.client('s3', region_name=region)
    s3_client.head_object(Bucket=bucket_name, Key=file_key)
    print(f"Verified that s3://{bucket_name}/{file_key} exists")
except Exception as e:
    print(f"Warning: Error verifying S3 file: {e}")

Found train.py in the current directory
Uploaded train.py to s3://blue-blood-data/code/train.py
Verified that s3://blue-blood-data/final_df.csv exists


In [ ]:
# Try a single model first before running hyperparameter tuning

# Define parameters for initial test run
test_epochs = 10
test_learning_rate = 0.001
test_lstm_units = 64
test_dropout_rate = 0.2

# Create a unique job name for the test run
test_job_name = f"lstm-med-test-e{test_epochs}-lr{test_learning_rate}-u{test_lstm_units}-d{int(test_dropout_rate*10)}"
print(f"Starting test job: {test_job_name}")

# Configure the TensorFlow estimator for the test run with correct settings
test_estimator = TensorFlow(
    entry_point="train.py",
    role=role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    framework_version="2.9",
    py_version="py39",
    hyperparameters={
        'epochs': test_epochs,
        'learning_rate': test_learning_rate,
        'lstm_units': test_lstm_units,
        'dropout_rate': test_dropout_rate,
        'batch_size': 1,
        'debug': True  # Set debug mode to limit epochs and simplify troubleshooting
    },
    job_name=test_job_name,
    disable_profiler=True,  # Disable profiler to avoid issues with smdebug
    debugger_hook_config=False,  # Disable debugger hooks
    environment={'SAGEMAKER_DISABLE_PROFILER': 'true'},  # Additional setting to disable profiler
    sagemaker_session=sagemaker_session,
    dependencies=["requirements.txt"]
)

test_estimator.fit({'train': input_data_s3_uri})


Starting test job: lstm-med-test-e10-lr0.001-u64-d2


[03/26/25 16:14:56] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=653772;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=965131;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=834440;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=258474;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[03/26/25 16:14:57] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=531042;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=280449;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=861686;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=200456;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             tensorflow-training-2025-03-26-21-14-56-167                                           

2025-03-26 21:14:59 Starting - Starting the training job...
2025-03-26 21:15:14 Starting - Preparing the instances for training...
2025-03-26 21:16:00 Downloading - Downloading the training image......
2025-03-26 21:16:56 Training - Training image download completed. Training in progress..2025-03-26 21:17:02.739224: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-03-26 21:17:02.739404: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2025-03-26 21:17:02.765397: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-03-26 21:17:04,833 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2025-03-26 21:17:04,844 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-26 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 │   dependencies=["requirements.txt"]                                                       │
│   31 )                                                                                           │
│   32                                                                                             │
│ ❱ 33 test_estimator.fit({'train': input_data_s3_uri})                                            │
│   34                                                                                             │
│                                                                                                  │
│ /opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py │
│ :167 in wrapper                                                                                  │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py │
│ :138 in wrapper                                                                                  │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:3 │
│ 46 in wrapper                                                                                    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                   

In [4]:
# Add this cell after your test_estimator.fit() call to view logs
from sagemaker.tensorflow.estimator import TensorFlow

# Get the logs for the training job
training_job_name = test_estimator.latest_training_job.job_name
print(f"Training job name: {training_job_name}")

# Stream the logs
logs = sagemaker_session.logs_for_job(training_job_name)
print("\nTraining logs:")
print(logs)

Training job name: tensorflow-training-2025-03-26-20-45-57-327
2025-03-26 20:48:26 Starting - Preparing the instances for training
2025-03-26 20:48:26 Downloading - Downloading the training image
2025-03-26 20:48:26 Training - Training image download completed. Training in progress.
2025-03-26 20:48:26 Uploading - Uploading generated training model
2025-03-26 20:48:26 Failed - Training job failed2025-03-26 20:48:06.217035: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-03-26 20:48:06.217201: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2025-03-26 20:48:06.242889: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-03-26 20:48:08,296 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2025-03-2